In [1]:
# Install Required Python Moules

!pip3 install google-api-python-client google-auth-httplib2 google-auth-oauthlib python-telegram-bot &>/dev/null

In [ ]:
# *****************************************







# =========================================
# Main Functions Starts from here
# =========================================








# *****************************************

In [2]:
import os
import shutil
import io
import pickle
import telegram
from re import search as re_search
from io import FileIO
from urllib.parse import parse_qs, urlparse
from os import makedirs, path as ospath, listdir, remove as osremove
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload





# Replace YOUR_TOKEN with your actual bot token
token = "5558586331:AAHcUlXjsECwp8UkreX7KgqelH0X_oXfTjc"

# Replace CHAT_ID with the chat ID of the recipient
chat_id = "-1001578391154"

# Replace FILE_PATH with the path to your media file
d_path = "/content/Downloads"
if not ospath.exists(d_path):
    makedirs(d_path)

# Replace THUMB_PATH with the path to your thumbnail file (optional)
thumb_path = "/content/thmb.jpg"

# Create a new Telegram bot instance using the bot token
bot = telegram.Bot(token=token)

print('Bot Variables Initialized !')

Bot Variables Initialized !


In [5]:
import os
import pickle
from googleapiclient.discovery import build


# create credentials object from token.pickle file
creds = None
if os.path.exists("/content/token.pickle"):
    with open("/content/token.pickle", "rb") as token:
        creds = pickle.load(token)
else:
    exit(1)

# create drive API client
service = build("drive", "v3", credentials=creds)

print('Service object was built for GDrive !')

Service object was built for GDrive !


In [6]:
from re import search as re_search
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError



# extract the file ID or folder ID from the link
def __getIdFromUrl(link: str):
    if "folders" in link or "file" in link:
        regex = r"https:\/\/drive\.google\.com\/(?:drive(.*?)\/folders\/|file(.*?)?\/d\/)([-\w]+)"
        res = re_search(regex, link)
        if res is None:
            raise IndexError("G-Drive ID not found.")
        return res.group(3)
    parsed = urlparse(link)
    return parse_qs(parsed.query)["id"][0]


# Get File Metadata
def __getFileMetadata(file_id):
    return (
        service.files()
        .get(fileId=file_id, supportsAllDrives=True, fields="name, id, mimeType, size")
        .execute()
    )

# enter the link for the file or folder that you want to download
link = input("Enter the Google Drive link for the file or folder: ")

file_id = __getIdFromUrl(link)

meta = __getFileMetadata(file_id)

d_name = meta['name']

d_fol_path = f"{d_path}/{d_name}"

print('GDrive Link was Processed !')

Enter the Google Drive link for the file or folder: https://drive.google.com/drive/u/0/folders/1nC4FEILQkqAU89rVRsTpuDL3KHKCGlq0
GDrive Link was Processed !


In [7]:
import io
import os
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from os import makedirs, path as ospath, listdir, remove as osremove



# A list for files inside a folder
def __getFilesByFolderId(folder_id):
    page_token = None
    files = []
    while True:
        response = (
            service.files()
            .list(
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                q=f"'{folder_id}' in parents and trashed = false",
                spaces="drive",
                pageSize=200,
                fields="nextPageToken, files(id, name, mimeType, size, shortcutDetails)",
                orderBy="folder, name",
                pageToken=page_token,
            )
            .execute()
        )
        files.extend(response.get("files", []))
        page_token = response.get("nextPageToken")
        if page_token is None:
            break
    return files



def __download_file(file_id, path):
    try:
        file = service.files().get(fileId=file_id, supportsAllDrives=True).execute()
    except HttpError as error:
        print("An error occurred: {0}".format(error))
        file = None
    if file is None:
        print(
            "Sorry, the specified file or folder does not exist or is not accessible."
        )
    else:
        if file["mimeType"].startswith("application/vnd.google-apps"):
            print(
                "Sorry, the specified ID is for a Google Docs, Sheets, Slides, or Forms document. You can only download these types of files in specific formats."
            )
        else:
            # Create a BytesIO stream to hold the downloaded file data.
            file_contents = io.BytesIO()

            # Download the file or folder contents to the BytesIO stream.
            request = service.files().get_media(fileId=file_id, supportsAllDrives=True)
            file_downloader = MediaIoBaseDownload(file_contents, request)
            done = False
            while done is False:
                status, done = file_downloader.next_chunk()
                # print(f"\rDownload progress: {int(status.progress() * 100)}%")
            file_contents.seek(0)

            # Save the downloaded file or folder to disk using its original name (if available).
            file_name = file.get("name", f"untitleddrivefile_{file_id}")
            file_name = os.path.join(path, file_name)
            with open(file_name, "wb") as handle:
                handle.write(file_contents.getbuffer())
            print(
                # f'\nThe file "{file_name}" has been successfully downloaded!'
            )


def __download_folder(folder_id, path):

    folder_meta = __getFileMetadata(folder_id)
    folder_name = folder_meta['name']
    if not ospath.exists(f"{path}/{folder_name}"):
        makedirs(f"{path}/{folder_name}")
    path += f"/{folder_name}"
    result = __getFilesByFolderId(folder_id)
    if len(result) == 0:
            return
    result = sorted(result, key=lambda k: k['name'])
    for item in result:
        file_id = item['id']
        shortcut_details = item.get('shortcutDetails')
        if shortcut_details is not None:
            file_id = shortcut_details['targetId']
            mime_type = shortcut_details['targetMimeType']
        else:
            mime_type = item.get('mimeType')
        if mime_type == "application/vnd.google-apps.folder":
            __download_folder(file_id, path)
        else:
            __download_file(file_id, path)










# Determine if the ID is of file or folder
if meta.get("mimeType") == "application/vnd.google-apps.folder":
    __download_folder(file_id, d_path)
    print('\n\nFolder Was Downloaded Successfully !')
else:
    if not ospath.exists(d_fol_path):
        makedirs(d_fol_path)
    __download_file(file_id, d_fol_path)
    print('\n\nFile Was Downloaded Successfully !')




















Folder Was Downloaded Successfully !


In [17]:
import os
import shutil


def create_zip(folder_path):
    folder_name = os.path.basename(folder_path)  # get folder name from folder path
    zip_file_path = folder_path  # create zip file path
    shutil.make_archive(
        zip_file_path, "zip", folder_path
    )  # create zip file by archiving the folder
    return zip_file_path + ".zip"  # return zip file path

    

z_file_path = create_zip(d_fol_path)

print('Folder Successfully Compressed !')

Folder Successfully Compressed !


In [18]:
import shutil

shutil.rmtree(d_fol_path)

print('Original Download dir deleted !')

Original Download dir deleted !


In [19]:
import os

def size_checker(file_path):
    
    # max_size = 2097152000  # 2 GB
    max_size = 20971520
    file_size = os.stat(file_path).st_size

    if file_size > max_size:

        print(f'File size is {(file_size)/(1024*1024)} MB, so Splitting')
        if not ospath.exists(d_fol_path):
            makedirs(d_fol_path)

        split_zipFile(file_path, max_size)

        return True
    else:

        print(f'File size is {(file_size)/(1024*1024)} MB, so Skipping splitting')
        return False

        


def split_zipFile(file_path, max_size):

    dir_path, filename = os.path.split(file_path)

    new_path = f"{d_fol_path}/{filename}"

    with open(file_path, "rb") as f:
        chunk = f.read(max_size)
        i = 1

        while chunk:
            # Generate filename for this chunk
            ext = str(i).zfill(3)
            output_filename = "{}.{}".format(new_path, ext)

            # Write chunk to file
            with open(output_filename, "wb") as out:
                out.write(chunk)

            print(f'\n{output_filename} Was created !')

            # Get next chunk
            chunk = f.read(max_size)

            # Increment chunk counter
            i += 1



leech = size_checker(z_file_path)

print('\nFile Successfully Checked for leeching !')

File size is 108.02307033538818 MB, so Splitting

/content/Downloads/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ]/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.001 Was created !

/content/Downloads/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ]/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.002 Was created !

/content/Downloads/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ]/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.003 Was created !

/content/Downloads/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ]/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.004 Was created !

/content/Downloads/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ]/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.005 Was created !

/content/Downloads/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ]/(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.006 Was created !

File Successfully Checked for leeching

In [21]:
import os
from os import makedirs, path as ospath, listdir, remove as osremove
import telegram



def get_file_type(file_path):
    name, extension = os.path.splitext(file_path)
    if extension in [".mp4", ".avi", ".mkv", ".mov", ".webm", ".m4v"]:
        video_extension_fixer(file_path)
        return "video"
    elif extension in [".mp3", ".wav", ".flac", ".aac", ".ogg"]:
        return "audio"
    elif extension in [".jpg", ".jpeg", ".png", ".gif"]:
        return "photo"
    else:
        return "document"


def video_extension_fixer(file_path):

    dir_path, filename = os.path.split(file_path)

    if filename.endswith(".mp4") or filename.endswith(".mkv"):
        pass
    # split the file name and the extension
    else:
        # rename the video file with .mp4 extension
        name, ext = os.path.splitext(filename)
        os.rename(
            os.path.join(dir_path, filename), os.path.join(dir_path, name + ".mp4")
        )
        print(f"{filename} was changed to {name}.mp4")



async def upload_file(file_path, type, file_name):

    # Upload the file
    try:

        if type == "video":

            sent = await bot.send_video(
                chat_id=chat_id,
                video=file_path,
                supports_streaming=True,
                width=480,
                height=320,
                caption=file_name,
                thumb=thumb_path,
            )
            print(f'\n{file_name} Sent !')

        elif type == "audio":

            sent = await bot.send_audio(
                chat_id=chat_id,
                audio=file_path,
                supports_streaming=True,
                caption=file_name,
                thumb=thumb_path,
            )
            print(f'\n{file_name} Sent !')

        elif type == "document":

            sent = await bot.send_document(
                chat_id=chat_id,
                document=file_path,
                caption=file_name,
                thumb=thumb_path,
            )
            print(f'\n{file_name} Sent !')

        elif type == "photo":

            sent = await bot.send_photo(
                chat_id=chat_id,
                photo=file_path,
                caption=file_name,
            )
            print(f'\n{file_name} Sent !')

        print(f'\nLOG: {sent}')

    except Exception as e:
        print(e)








if leech: # File was splitted

    if ospath.exists(z_file_path):
      os.remove(z_file_path) # Delete original Big Zip file
    print('Big Zip File Deleted !')
    print('\n\n Now uploading multiple splitted zip files.............')

    dir_list = os.listdir(d_fol_path)

    for dir_path in dir_list:

        short_path = os.path.join(d_fol_path,dir_path)
        file_type = get_file_type(short_path)
        file_name = os.path.basename(short_path)
        # print(dir_path)
        await upload_file(short_path,file_type,file_name)

else:

    print('\nNow uploading the zip file..........................')

    file_type = get_file_type(z_file_path)
    file_name = os.path.basename(z_file_path)
    await upload_file(z_file_path,file_type,file_name)


print('\n\n All files successfully uploaded !')

Big Zip File Deleted !


 Now uploading multiple splitted zip files.............

(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.003 Sent !

LOG: Message(caption='(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.003', caption_entities=(MessageEntity(length=16, offset=36, type=<MessageEntityType.MENTION>),), channel_chat_created=False, chat=Chat(id=-1001578391154, title='Bot DUMPS 🗑️', type=<ChatType.CHANNEL>), date=datetime.datetime(2023, 3, 8, 6, 15, 48, tzinfo=<UTC>), delete_chat_photo=False, document=Document(file_id='BQACAgUAAx0EXhRWcgACBAdkCCgUqnvzIa2N-pgRwrvmGfU5yQACzAgAAhCQQFT4cUkRRDsENC4E', file_name='(ISC)² CAP Fundamentals - [Telegram @Myhackersworld2 ].zip.003', file_size=20971520, file_unique_id='AgADzAgAAhCQQFQ', thumb=PhotoSize(file_id='AAMCBQADHQReFFZyAAIEB2QIKBSqe_MhrY36mBHCu-YZ9TnJAALMCAACEJBAVPhxSRFEOwQ0AQAHbQADLgQ', file_size=10350, file_unique_id='AQADzAgAAhCQQFRy', height=180, width=320)), group_chat_created=False, message_id=1031, sender_chat